## Abstract Base Classes (ABCs)

### ABCs
* first defined in Python Enhancement Proposal 3119
* Python standard library: abc
* Python is not Java, C++, C sharp, it is much more flexible in terms of abstract classes

### Abstract Base Classes
* base: it is the target (base class) of an inheritnce relationship from a subclass
* abstract: the class can not be instantiated in isolation
* an abstract base class defines an interface which derived classes must implement 
* what is an interface? (interface definition) 
  + the base class defines the interface for clients of any and all subclasses
* Liskov substituability
  + subclasses, from the point of client code should be interchangable.
    + client code developed against an abstract interface should not require knowledge of specific concrete types, only of their capability as promised by the abstract base class
  + (client) code relying only on the base class does not need to be modified for alternative subclasses
* although providing interfaces, different from pure interfaces such as in Java, abstract class can also contain implementation code that can be shared by all derived classes

#### Duck typing
* why do we need to define named interfaces when we have duck typing?
  + determining whether a particular object supports the required interface before exercising that inteface can be quite awkward
  + for interfaces implementing many methods, to check whether or not an object has implemented all of them will not be conveient and efficient
* what is duck typing?
  + when I see a bird that walks like a duck and swims like a duck and quacks like a duck, I call that bird a duck
  + in Python, this is ture both in theory and practice
 

#### Abastract Base Classes in Python
* servs two purposes
  + provides specification
    + ABCs are effective for specifying interface protocols
  + detection
    + ABCs can be used to detect conforming objects
      + provide a means for easily determining whether an abitrary class or instance meets the requirements of a specific protocol
* base classes in Python
  + in the following code, we showed that list is a subclass of MutableSequence, and object, but MutableSequence is no where in list's baseclasses list, and even its transitive bases from mro
  + in the code example, we showed the 5 methods that a MutableSequence must implement out of the total 16. Note that the remaining 11 methods are implemented, but may not be the most efficient, since they only rely on the protocol without any knowledge of the concrete classes
  + The way issublist works is by checking its metaclass for a special method deunder subclasschek.
    + if this method is available, then call this method with the subclass name, for example 
    
    ```python
    issubclass(list, MutableSequence)
    
    # the same as
    if hasattr(type(MutableSequnece), "__subclasscheck__"):
        return type(MutableSequence).__subclasscheck__(list)   
    ```
    + it is upto the metaclass of MutableSequence to determine whether or not list is a subclass of MutableSequence, rather than list being required to know that MutableSequence is one of its base classes
    + this allows list to be a subclass of MutableSequence without MutableSequence being the target of an inheritance relationship
    + we can say list is a virtual subclass of MutableSequence and MutableSequence is a virtual base class of list

In [1]:
from collections.abc import (MutableSequence)

print(issubclass(list, object))
print(issubclass(list, MutableSequence))
print(list.__bases__)
print(list.__mro__)

True
True
(<class 'object'>,)
(<class 'list'>, <class 'object'>)


In [2]:
# error message of this command shows the 5 abstract methods 
# we need to implement, out of the all 16 methods where 11 has been implemented
ms = MutableSequence()

TypeError: Can't instantiate abstract class MutableSequence with abstract methods __delitem__, __getitem__, __len__, __setitem__, insert

#### Defining \_\_subclasscheck\_\_ in practice
* starting from two sword subclass, we created a ABC for these two. These two classes have the identical interfaces
* after adding a metaclass for Sword class, and implement dunder subclasscheck method, issubclass() returns True, even though the two concrete classes didn't inherit Sword class.
* also implemented dunder instancecheck, which calls issubclass that finally calls the dunder subclasscheck method of the metaclass
* the implementation in dunder subclasscheck doesn't check for subclassing by inheritance. It is a test if the subclasses are exchangable by following the certain function signatures. Therefore, it is a test for Replaces rather than extends
  + this purely duck-tying checking will return false if a class inherits Sword. To fix this, we can fall back to normal dunder subclasscheck if the structure/duck-type check gives us False results. By doing this in version 3, issubclass returns true for both duck-type and inheritance subclasses.

In [14]:
# version 1. two concerete classes with identical interfaces 
# but no inheritance relationship
class Sword:
    """Abstract Base Class"""   

class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)       
        

In [15]:
# we first check that both concrete classes work fine 
b = BroadSword()
print(b.swipe())

b = SamuraiSword()
print(b.swipe())

# There is no inheritance relationship
print(issubclass(BroadSword, Sword))

print(issubclass(SamuraiSword, Sword))

Swoosh! BroadSword
None
Slice! SamuraiSword
None
False
False


In [18]:
# version 2. implement metaclass for abstract class Sword
# to define the inheritance relationship

class SwordMeta(type):
    
    # this function will call __subclasscheck__ under the hood
    # issubclass will call __subclasscheck__ from the metaclass 
    # of cls(here cls==Sword, metaclass==SwordMeta)
    def __instancecheck__(cls, instance):
        return issubclass(type(instance), cls)
    
    
    def __subclasscheck__(cls, subclass):
        return (
            hasattr(subclass, "swipe") and callable(subclass.swipe)
            and
            hasattr(subclass, "sharpen") and callable(subclass.sharpen)
        )



class Sword(metaclass=SwordMeta):
    """Abstract Base Class"""   

class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass

In [20]:
# There is inheritance relationship
# after adding metaclass for Sword
print(issubclass(BroadSword, Sword))
print(issubclass(SamuraiSword, Sword))
print(issubclass(Rifle, Sword))

# isinstance returns true
samurai_sword = SamuraiSword()
isinstance(samurai_sword, Sword)

# inheritance relationship is ignored
issubclass(Sabre, Sword)

True
True
False


False

In [25]:
# version 2. implement metaclass for abstract class Sword
# to define the inheritance relationship

class SwordMeta(type):
    
    # this function will call __subclasscheck__ under the hood
    # issubclass will call __subclasscheck__ from the metaclass 
    # of cls(here cls==Sword, metaclass==SwordMeta)
    def __instancecheck__(cls, instance):
        return issubclass(type(instance), cls)
    
    
    def __subclasscheck__(cls, subclass):
        if (
            hasattr(subclass, "swipe") and callable(subclass.swipe)
            and
            hasattr(subclass, "sharpen") and callable(subclass.sharpen)
        ):
            return True
        return super().__subclasscheck__(subclass)



class Sword(metaclass=SwordMeta):
    """Abstract Base Class"""   

class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass        

In [26]:
print(issubclass(BroadSword, Sword))
print(issubclass(SamuraiSword, Sword))

print(issubclass(Rifle, Sword))
print(issubclass(Sabre, Sword))

True
True
False
True


#### Virtual Base Classes from collections.abc
* The duck-type subclass checking has been used in some of the collection.abc base classes, including Sized
* implementation of dunder len function is sufficient to be considered a subclass of Sized base class
* another example is iterable and iterator, which requires implementation of dunder iter and dunder next to be considered their sub classes
* Using virtual base classes such as those from collections.abc to test for the implementation of protocols 
* ABCs in Python gives a way to centralize structural or duck-type cheks in a natural way by extending the issubclass and isinstance mechanisms  

In [27]:
# example of Sized subclasses
from collections.abc import Sized

class SixPack:
    
    # __len__ will be called by built-in len() function
    def __len__(self):
        return 6


In [29]:
s = SixPack()
print(len(s))
print(issubclass(SixPack, Sized))

6
True


In [31]:
# example of iterable and iterateor subclasses

class EndlessScreaming:
    def __iter__(self):
        return self
    
    def __next__(self):
        return "abc"
    
from collections.abc import Iterable, Iterator

i = EndlessScreaming()
print(isinstance(i, Iterator))
print(isinstance(i, Iterable))
print(issubclass(Iterator, Iterable))

True
True
True


#### non-transitive subclass relationship
* The base clase-subclass relationship may not be symmetric. Meanning that a subclass may consider another class as its base class but that base class may not consider the sub class as its sub class, due to the duck-typing. In addition, the base-sub class relationship may not be transitive
* transitive 
  + being or relating to a relation with the property that if the relation holds between a first and second elements, and between the second and third elements, it holds between the first and third elements
  + in Python, if C is a subclass of B, and B is a subclass of A, that doesn't necessarily mean C is a subclass of A
    + in the following example, object is a subclass of Hashable, and list is a subclass of object, but list is not a subclass of Hashable
    + this is becauase, as a Mutable collection, disables hashing by setting the special \_\_hash\_\_ method inhrtited from object to None, which is checked by dunder subclass\_check method of Hashable 

In [32]:
from collections.abc import Hashable

print(issubclass(object, Hashable))
print(issubclass(list, object))
print(issubclass(list, Hashable))

True
True
False


In [34]:
list.__hash__ is None

True

#### method resolution with virtual base classes
* virtual base classes don't play a role in method resolution
* in the following code example, we added a method, Thrust in Sword class, and even though a BoradSword object is an instance of Sword, we will not be able to invoke thrust method from a BroadSword object, since Sword class is not on mro of BroadSword class instance objects

In [37]:
class SwordMeta(type):
    
    # this function will call __subclasscheck__ under the hood
    # issubclass will call __subclasscheck__ from the metaclass 
    # of cls(here cls==Sword, metaclass==SwordMeta)
    def __instancecheck__(cls, instance):
        return issubclass(type(instance), cls)
    
    
    def __subclasscheck__(cls, subclass):
        if (
            hasattr(subclass, "swipe") and callable(subclass.swipe)
            and
            hasattr(subclass, "sharpen") and callable(subclass.sharpen)
        ):
            return True
        return super().__subclasscheck__(subclass)



class Sword(metaclass=SwordMeta):
    """Abstract Base Class""" 
    
    def thrust(self):
        print("Thrust!", type(self).__name__)

class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass        

In [39]:
broad_sword = BroadSword()
print(isinstance(broad_sword, Sword))
print(BroadSword.__mro__)
broad_sword.thrust()

True
(<class '__main__.BroadSword'>, <class 'object'>)


AttributeError: 'BroadSword' object has no attribute 'thrust'

### Standard Library Support for ABCs

#### Replacing Metaclasses with Abstract Base Classes
* Custom metaclass 
  + implements dunder subclasschek and dunder instancecheck
  + tricky to get right
  + onerous to implement a metaclass
  + metaclasses are not widely understood
* Standard library support for ABCs include
  + abc module
  + ABCMeta metaclass
  + ABC base class
  + @register class decorator
  + @abstractmethod decorator
* the example code shows how to comvert the SwordMeta to ABCMeta class by implementing dunder subclasshook method
  + we import ABCMeta class
    + implements reliable dunder subclasscheck and dunder instancecheck methods and other handy capabilities
    + dunder subclasscheck and dunder instancecheck will delegate the dunder subclasshook method in Sword class for subclass and instance check
  + set metacalss of Sword class to ABCMeta, instead of SwordMeta 
  + implement dunder subclasshook in Sword abstract class
  + in fact, all Python objects have dunder subclasshook method, which accepts the potential subclass as its only argument
    + the method returns True, False, or NotImplemented    
      
  ```python
    # test if sub is a virtual subclass of the base
    Base.__subclasshook__(sub)
  ```  
    + for testing with a returned value of NotImplemented, the inheritance subclass relationship can be tested following the MRO mechanism. In the example code, we tested that int is not a virtual subclass of object, but is subclass of object from the inheritance point of view
      + issubclass test covers both virtual and inheritance subclass relationship. If either or them is True, it returns Ture. subclasshook returns only test for virtual subclass
    

In [45]:
# int is not a virtual subclass of object
print(object.__subclasshook__(int))

# int is a subclass of object
print(issubclass(int, object))
print(int.__mro__)

NotImplemented
True
(<class 'int'>, <class 'object'>)


In [44]:
print(int.__mro__)

(<class 'int'>, <class 'object'>)


In [46]:
# using ABCMeta to implement virtual subclasses
from abc import ABCMeta

class Sword(metaclass=ABCMeta):
    """Abstract Base Class""" 
    
    @classmethod
    def __subclasshook__(cls, subclass):
        return (
            hasattr(subclass, "swipe") and callable(subclass.swipe)
            and
            hasattr(subclass, "sharpen") and callable(subclass.sharpen)
        )
            
    
    def thrust(self):
        print("Thrust!", type(self).__name__)

class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass        

In [47]:
print(issubclass(BroadSword, Sword))
print(issubclass(SamuraiSword, Sword))

print(issubclass(Rifle, Sword))
print(issubclass(Sabre, Sword))

True
True
False
False


#### Virtual subclass registration
* in addition to implementing dunder subclasshook(cls, subclass) method in abstract base class, we can directly register a class as a virtual subclass using registermeta method
* in the following example code, we create an abstract base class, Text using ABCMeta as its meta class, and register the built-in str class as its virtual subclass by calling the Text class method register
  + note that this method returns the registered subclass object as the return value
* We can also register a class as a virtual subclass of another class using class decorator, @base.register in its definition
  + for example, we can define a new class Prose, and register it as a virtual subclass of Text
  + this is how Python internally define float and int as virual subclasses of Real and Integral, respectively
    + look at the Python source code of Real and Integral, we can find the following source code for subclass registry:
    
    ```python
    Real.register(float)
    Integral.register(int)
    ```

In [53]:
from abc import ABCMeta

class Text(metaclass=ABCMeta):
    pass

# register a subclass of Text by calling its register method
print(Text.register(str))
print(issubclass(str, Text))

<class 'str'>
True


In [56]:
# define a class Prose and 
# register it as a virtual subclass of Text

@Text.register
class Prose:
    pass

print(issubclass(Prose, Text))

True


In [57]:
# internally, Python define virtual subclass relationship 
# between int and Integeral, and between float and Real
from numbers import Real, Integral

print(issubclass(float, Real))
print(issubclass(int, Integral))

True
True


#### Combing Subclass Detection and Registration
* we can combine subclass detection (dunder subclasshook) and register mechanisms together
* dunder subclasshook takes the precedence of subclass registry
* returning true or false from dunder subclasshook is taken as a definite answer. If you want to honor registered subclasses, you must return NotImplemented to trigger lookup of registered subclasses
* in the code example, we defined a LightSabre class, that only implements swipe() method, without sharpen() method. Therefore, will not pass the criteria of dunder subclasshook method. We just register it as a virtual subclass of Sword
  + we do this by first register it as a subclass of Sword using @Sword.register class decorator
  + we then change the dunder subclasshook method to return NotImplemented when the check conditions don't pass, so that to trigger registry lookup if the conditions are not met
   + the consequence of using both detection and registration making the abstract base class Sword not useful since there is no guarantee that virtual subclasses will honor the contract to implement all methods in Sword class, which is the basic funtion of this class 
* Disadvantages of using subclass registration:
  + inadvertently weaken the base class concept
  + registration can bypass interfact detection

In [60]:
# using both subclass detection and registration
from abc import ABCMeta

class Sword(metaclass=ABCMeta):
    """Abstract Base Class""" 
    
    @classmethod
    def __subclasshook__(cls, subclass):
        return (
            (
                hasattr(subclass, "swipe") and callable(subclass.swipe)
                and
                hasattr(subclass, "sharpen") and callable(subclass.sharpen)
            )
            or
            NotImplemented
        ) 
    
    def thrust(self):
        print("Thrust!", type(self).__name__)

@Sword.register
class LightSabre:
    
    def swipe(self):
        print("FFFFkrrrshhzzwooooom..woom..woooom..", type(self).__name__)
        
class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass        

In [62]:
print(issubclass(BroadSword, Sword))
print(issubclass(SamuraiSword, Sword))

print(issubclass(Rifle, Sword))
print(issubclass(LightSabre, Sword))

True
True
False
True


#### ABC Base class
* the abc module contains an abstract base class ABC
* it is an abstract class using ABCMeta as its metaclass
* this makes it easier to declare abstract base classes without having to put the metaclass mechanism on show
* inheritance from ABC is a preferred way to signal that a class is an abstract base class
  + it is clearer and more widely understood than metaclasses
  + facilitates overriding dunder subclasshook method
  + It brings other benefits too to help us to define abstract base classes based on ABC with ABCMeta, such as method deocrators
* in the following code, we simplified Sword class
  + we don't need to expose ABCMeta class, and we don't need to even import it
  + we only need to import ABC class and make Sword class to inherit ABC

In [63]:
# using ABC to simplify Sword abstract base class
# we don't need to expose ABCMeta
from abc import ABC

class Sword(ABC):
    """Abstract Base Class""" 
    
    @classmethod
    def __subclasshook__(cls, subclass):
        return (
            (
                hasattr(subclass, "swipe") and callable(subclass.swipe)
                and
                hasattr(subclass, "sharpen") and callable(subclass.sharpen)
            )
            or
            NotImplemented
        ) 
    
    def thrust(self):
        print("Thrust!", type(self).__name__)

@Sword.register
class LightSabre:
    
    def swipe(self):
        print("FFFFkrrrshhzzwooooom..woom..woooom..", type(self).__name__)
        
class BroadSword:
    
    def swipe(self):
        print("Swoosh!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
        
class SamuraiSword:
    
    def swipe(self):
        print("Slice!", type(self).__name__)
        
    def sharpen(self):
        print("Shink!", type(self).__name__)
        
class Rifle:
    
    def fire(self):
        print("Bang!", type(self).__name__)
        
class Sabre(Sword):
    pass        

In [64]:
print(issubclass(BroadSword, Sword))
print(issubclass(SamuraiSword, Sword))

print(issubclass(Rifle, Sword))
print(issubclass(LightSabre, Sword))

True
True
False
True


#### abstractmethod decorator
* 